In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\n\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [2]:
from trulens_eval.keys import *

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


In [3]:
# LLama Index starter example from: https://gpt-index.readthedocs.io/en/latest/getting_started/starter_example.html

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('llama_index/data').load_data()
index = GPTVectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
# response = query_engine.query("What did the author do growing up?")
# print(response)

In [4]:
from trulens_eval import TruLlama, Tru, Query, Feedback
from trulens_eval import tru_feedback

tru = Tru()
Tru().reset_database()

proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent.parent)
thread = Tru().start_evaluator(restart=True)

Force stopping dashboard ...
Starting dashboard ...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking for things to do. Stop me with `tru.stop_evaluator()`.

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
import numpy as np

# Construct feedback functions.

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.model.retriever.retrieve.rets[:].node.text
).aggregate(np.min)

feedbacks = [f_lang_match, f_qa_relevance, f_qs_relevance]

l = tru.Llama(engine=query_engine, feedbacks=feedbacks)

huggingface api: 0requests [00:00, ?requests/s]

openai api: 0requests [00:00, ?requests/s]

✅ chain chain_hash_a11d1654555280a1e757beda5d0bb9bb -> default.sqlite
✅ feedback def. feedback_definition_hash_13c1ee85757cb8db89f2a97f7ff41892 -> default.sqlite
✅ feedback def. feedback_definition_hash_1c499dc13210cf0a8084c4e91df0af30 -> default.sqlite
✅ feedback def. feedback_definition_hash_229a88b0e29d99f89a98468052513c56 -> default.sqlite



  You can now view your Streamlit app in your browser.

  Network URL: http://192.168.1.60:8501
  External URL: http://73.222.135.127:8501

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY
Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.L

In [ ]:
list(l.instrumented())

In [ ]:
res, record = l.query_with_record("What did the author do growing up?")

In [ ]:
record.dict()